# Data Visualization Dashboard Project

## Import packages

In [1]:
import eurostat
import pandas as pd
import openpyxl


In [2]:
import plotly.graph_objects as go
import numpy as np

In [3]:
data_out = '../data/'

## Codes

In [4]:
siec_codes = eurostat.get_dic('siec')

geo_eu = [
'BE','BG','CZ','DK','DE',
'EE','IE','EL','ES','FR',
'HR','IT','CY','LV','LT',
'LU','HU','MT','NL','AT',
'PL','PT','RO','SI','SK',
'FI','SE','IS','LI','NO',
'CH',
]

geo_codes = {
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'CZ': 'Czechia',
    'DK': 'Denmark',
    'DE': 'Germany',
    'EE': 'Estonia',
    'IE': 'Ireland',
    'EL': 'Greece',
    'ES': 'Spain',
    'FR': 'France',
    'HR': 'Croatia',
    'IT': 'Italy',
    'CY': 'Cyprus',
    'LV': 'Latvia',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'HU': 'Hungary',
    'MT': 'Malta',
    'NL': 'Netherlands',
    'AT': 'Austria',
    'PL': 'Poland',
    'PT': 'Portugal',
    'RO': 'Romania',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'FI': 'Finland',
    'SE': 'Sweden',
    'IS': 'Iceland',
    'LI': 'Liechtenstein',
    'NO': 'Norway',
    'CH': 'Switzerland',
    'UK': 'United Kingdom'
}

x = ['R5110-5150_W6000RI',
 'R5200',
 'R5300',
 'RA000',
 'RA200',
 'RA410',
 'RA500_5160',
 'RA600',
 'W6210'
]

rs_codes = [
 'R5110-5150_W6000RI',
 'R5200', 
 'R5300', 
 'RA100',
 'RA200',
 'RA300',
 'RA410',
 'RA420', 
 'RA500_5160', 
 'RA600',
 'W6210',
# 'RA000',
           ]



## Get SIEC Timeseries

In [5]:
es_code = 'NRG_IND_COTD'
data = eurostat.get_data_df(str.lower(es_code), flags=False)

In [6]:
## Rename columns:
data.rename(columns={'geo\\time': 'Country Code', 'siec':'SIEC Code'}, inplace=True)

In [7]:
# Keep only selected countries
data = data.loc[data['Country Code'].isin(geo_eu),:]

In [8]:
# Remove extra rows in nrg_bal
# Keep only:
# Gross final consumption - energy use - overall target calculation without aviation cap adjustment 
# [GFC_E_OT_NAVICAPA]

data = data.loc[data['nrg_bal']=='GFC_E_OT_NAVICAPA',:]


In [9]:
# Remove unneccessary cols 
data.drop(columns=['nrg_bal','unit'], inplace=True)

In [10]:
data.reset_index(drop=True, inplace=True)

In [11]:
data['Country'] = data['Country Code']
data['Country'] = data['Country'].map(geo_codes)

In [12]:
data['SIEC'] = data['SIEC Code']
data['SIEC'] = data['SIEC'].map(siec_codes)

In [13]:
data.to_csv(data_out + 'renewables_percent_timeseries.csv', index=False)

## Get Energy Flow Data 

In [14]:
sk_code = 'NRG_BAL_SD'
skdata = eurostat.get_data_df(str.lower(sk_code), flags=False)

In [15]:
## AAS = Available from all sources
renewable_sources2020 = skdata.loc[skdata['geo\\time'].isin(geo_eu)].loc[skdata['nrg_bal']=='AAS',['geo\\time', 'nrg_bal', 'siec', 2020]]
renewable_sources2020.rename(columns={'geo\\time': 'Country Code'}, inplace=True)
renewable_sources2020.dropna(inplace=True)


#renewable_sources2020 = renewable_sources2020.loc[renewable_sources2020['siec'].isin(rs_codes)]

renewable_sources2020['SIEC'] = renewable_sources2020['siec'].map(siec_codes)
renewable_sources2020['Country'] = renewable_sources2020['Country Code'].map(geo_codes)

renewable_sources2020 = renewable_sources2020.rename(columns={
                                         2020: 'Consumption in KTOE',
                                         'siec': 'SIEC Code'
                                        }).reset_index(drop=True).drop(columns=['nrg_bal'])




In [16]:
siec_parent = {
    'RA000' : 'X',
    'RA600' : 'Other Renewable',
    'R5300' : 'Biomass',
    'R5200' : 'Biomass',
    'RA500_5160': 'Other Renewable',
    'R5110-5150_W6000RI' : 'Other Renewable',
    'W6210' : 'Other Renewable',
    'RA420' : 'Solar',
    'RA410' : 'Solar',
    'RA100' : '', 
    'RA200' : '', 
    'RA300' : '', 
    
    
    'E7000': '',
    
    'O4000': 'X',
    'O4100_TOT' : 'Oil and petroleum products',
    'O4200' : 'Oil and petroleum products',
    'O4652XR5210B' : 'Oil and petroleum products',
    'O4661XR5230B' : 'Oil and petroleum products',
    'O4671XR5220B' : 'Oil and petroleum products',
    'O4680' : 'Oil and petroleum products',
    'PP_OTH' : 'Oil and petroleum products',
    'O4200-4500': 'Oil and petroleum products',
    
    'SFF_P1000': 'X',
    'SFF_OTH' : 'Solid fossil fuels and peat',
    'C0121' : 'Solid fossil fuels and peat',
    'C0129' : 'Solid fossil fuels and peat',
    'C0220' : 'Solid fossil fuels and peat',
    'C0311' : 'Solid fossil fuels and peat',
    
    'W6100_6220': 'X',
    'W6220': 'Other non-renewable',
    
    'G3000_C0350-370': 'X',
    'G3000' : 'Gas',
    'C0350-0370' : 'Gas',
    
    
    'H8000' : 'Other non-renewable',
    'N900H' : 'Other non-renewable',
    'E7000' : 'Other non-renewable',
    'W6100' : 'Other non-renewable',

    'TOTAL' : 'X'
    
}




In [17]:
renewable_sources2020['Parent'] = renewable_sources2020['SIEC Code'].replace(siec_parent)
renewable_sources2020['Renewable'] = renewable_sources2020['SIEC Code'].isin(rs_codes)

renewable_sources2020.loc[renewable_sources2020['Renewable']==True,'Renewable'] = 'Renewable'
renewable_sources2020.loc[renewable_sources2020['Renewable']==False,'Renewable'] = 'Non-renewable'

In [18]:
renewable_sources2020.loc[renewable_sources2020['SIEC Code'] == renewable_sources2020['Parent'], 'SIEC'].unique()

array([], dtype=object)

In [19]:
#renewable_sources2020

In [20]:
df_be = pd.DataFrame(renewable_sources2020.loc[renewable_sources2020['Country Code']=='SE']
             .groupby(['SIEC', 'SIEC Code', 'Parent', ]).sum()).reset_index().sort_values(by=['Parent'])


In [21]:
df_be.loc[df_be['SIEC']=='Total']

,SIEC,SIEC Code,Parent,Consumption in KTOE
29,Total,TOTAL,X,68806.9


Solid fossil fuels and peat	SFF_P1000
Gas	G3000_C0350-370
Oil and petroleum products	O4000	
Renewables and biofuels	RA000	


G3000_C0350-370	
O4000
RA000
SFF_P1000
= 80055.7

In [22]:
#         other nonren + X
# 90577.7 - ( 11349.3 + 80055.7 )

In [23]:
# 90577.7

In [24]:
renewable_sources2020 = renewable_sources2020.loc[renewable_sources2020['Parent']!='X']

In [25]:
renewable_sources2020['Sunburst_Parent'] = renewable_sources2020['Parent']
renewable_sources2020['Sunburst_SIEC'] = renewable_sources2020['SIEC']


In [26]:
renewable_sources2020.loc[renewable_sources2020['Sunburst_Parent']=='', 'Sunburst_Parent'] = renewable_sources2020.loc[renewable_sources2020['Sunburst_Parent']=='', 'SIEC']


In [27]:
renewable_sources2020.loc[renewable_sources2020['Parent']=='', 'Sunburst_SIEC'] = '' #= renewable_sources2020.loc[renewable_sources2020['Sunburst_Parent']=='', 'SIEC']


In [28]:
colors = ['#363537', '#fcfcfc', '#bee9e8', '#62b6cb', '#1b4965', '#ffef84', '#c3d37a', '#86b66f', '#0c7c59']


In [29]:
renewable_sources2020#['Sunburst_Parent'].unique()


,Country Code,SIEC Code,Consumption in KTOE,SIEC,Country,Parent,Renewable,Sunburst_Parent,Sunburst_SIEC
0,AT,C0121,1208.7,Coking coal,Austria,Solid fossil fuels and peat,Non-renewable,Solid fossil fuels and peat,Coking coal
1,BE,C0121,1143.2,Coking coal,Belgium,Solid fossil fuels and peat,Non-renewable,Solid fossil fuels and peat,Coking coal
2,CZ,C0121,2429.4,Coking coal,Czechia,Solid fossil fuels and peat,Non-renewable,Solid fossil fuels and peat,Coking coal
3,DE,C0121,7096.0,Coking coal,Germany,Solid fossil fuels and peat,Non-renewable,Solid fossil fuels and peat,Coking coal
4,ES,C0121,475.8,Coking coal,Spain,Solid fossil fuels and peat,Non-renewable,Solid fossil fuels and peat,Coking coal
...,...,...,...,...,...,...,...,...,...
822,PL,W6220,408.5,Non-renewable municipal waste,Poland,Other non-renewable,Non-renewable,Other non-renewable,Non-renewable municipal waste
823,PT,W6220,87.7,Non-renewable municipal waste,Portugal,Other non-renewable,Non-renewable,Other non-renewable,Non-renewable municipal waste
824,RO,W6220,2.4,Non-renewable municipal waste,Romania,Other non-renewable,Non-renewable,Other non-renewable,Non-renewable municipal waste
825,SE,W6220,892.1,Non-renewable municipal waste,Sweden,Other non-renewable,Non-renewable,Other non-renewable,Non-renewable municipal waste


In [30]:
#gparent = pd.DataFrame(s_.groupby(['Sunburst_Parent', 'Renewable']).size()).reset_index().drop(columns=[0])

#gparent.loc[gparent['Sunburst_Parent']=='Gas', 'Renewable'].values[0]

In [31]:
s_ = renewable_sources2020.loc[(renewable_sources2020['Country Code']=='AT'),['Sunburst_SIEC','Sunburst_Parent','Consumption in KTOE', 'Renewable']]
#gparent = pd.DataFrame(s_.groupby(['Sunburst_Parent', 'Renewable']).size()).reset_index().drop(columns=[0])

#[ gparent.loc[gparent['Sunburst_Parent']==_, 'Renewable'].values[0] for _ in (s_['Sunburst_Parent'].unique()) ]

#[ _ for _ in (s_['Sunburst_Parent'].unique()) ]

s_['Renewable'].unique()

array(['Non-renewable', 'Renewable'], dtype=object)

In [32]:
s_['Renewable'].unique()

array(['Non-renewable', 'Renewable'], dtype=object)

In [33]:
s_['Sunburst_Parent'].unique()

array(['Solid fossil fuels and peat', 'Other non-renewable', 'Gas',
       'Oil and petroleum products', 'Other Renewable', 'Biomass',
       'Hydro', 'Geothermal', 'Wind', 'Solar'], dtype=object)

In [34]:
#s_labels = s_['Renewable'].unique() + s_['Sunburst_Parent'].unique() + s_['Sunburst_SIEC'].values 
#s_labels = 
#s_['Renewable'].unique()
type(s_['Sunburst_Parent'].unique())
#type(s_['Sunburst_SIEC'].values )

#s_labels = s_['Renewable'].unique().tolist() + s_['Sunburst_Parent'].unique().tolist() + s_['Sunburst_SIEC'].values.tolist()
#s_labels

numpy.ndarray

In [35]:
#s_.groupby('Renewable').sum().reset_index()

#s_.loc[s_['Renewable']=='Renewable','Consumption in KTOE'].sum()

In [36]:

renewable_sources2020.to_csv(data_out + 'renewable_sources_2020.csv', index=False)